## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Severo-Kurilsk,50.6789,156.1250,27.82,77,80,13.76,RU,2022-01-31 06:34:53,light snow
1,1,Erice,38.0370,12.5865,48.67,82,75,16.11,IT,2022-01-31 06:34:53,broken clouds
2,2,Lompoc,34.6391,-120.4579,55.13,88,69,13.00,US,2022-01-31 06:34:54,broken clouds
3,3,Tuktoyaktuk,69.4541,-133.0374,-0.40,78,100,23.02,CA,2022-01-31 06:34:54,light snow
4,4,Bosaso,11.2842,49.1816,76.98,78,20,5.75,SO,2022-01-31 06:34:54,few clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_temp_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_temp_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
4,4,Bosaso,11.2842,49.1816,76.98,78,20,5.75,SO,2022-01-31 06:34:54,few clouds
6,6,Salalah,17.0151,54.0924,75.29,60,0,2.30,OM,2022-01-31 06:34:55,clear sky
12,12,Hithadhoo,-0.6000,73.0833,82.60,70,5,10.20,MV,2022-01-31 06:34:57,clear sky
14,14,Sur,22.5667,59.5289,77.76,48,0,3.94,OM,2022-01-31 06:34:58,clear sky
15,15,Rikitea,-23.1203,-134.9692,77.32,62,100,4.92,PF,2022-01-31 06:34:58,overcast clouds
18,18,Sinnamary,5.3833,-52.9500,75.15,89,89,9.22,GF,2022-01-31 06:35:00,overcast clouds
20,20,Sao Jose Da Coroa Grande,-8.8978,-35.1478,77.49,83,11,5.64,BR,2022-01-31 06:35:00,few clouds
25,25,Atuona,-9.8000,-139.0333,78.22,75,8,15.68,PF,2022-01-31 06:35:02,clear sky
28,28,Lorengau,-2.0226,147.2712,81.37,80,100,15.39,PG,2022-01-31 06:35:03,overcast clouds
30,30,Saldanha,-33.0117,17.9442,77.05,58,36,1.77,ZA,2022-01-31 06:35:04,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_temp_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Date                   0
Current Description    0
dtype: int64

In [7]:
preferred_temp_df.count()

City_ID                170
City                   170
Lat                    170
Lng                    170
Max Temp               170
Humidity               170
Cloudiness             170
Wind Speed             170
Country                170
Date                   170
Current Description    170
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_temp2_df = preferred_temp_df.dropna()
preferred_temp2_df.count()

City_ID                170
City                   170
Lat                    170
Lng                    170
Max Temp               170
Humidity               170
Cloudiness             170
Wind Speed             170
Country                170
Date                   170
Current Description    170
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_temp2_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Bosaso,SO,76.98,few clouds,11.2842,49.1816,
6,Salalah,OM,75.29,clear sky,17.0151,54.0924,
12,Hithadhoo,MV,82.60,clear sky,-0.6000,73.0833,
14,Sur,OM,77.76,clear sky,22.5667,59.5289,
15,Rikitea,PF,77.32,overcast clouds,-23.1203,-134.9692,
18,Sinnamary,GF,75.15,overcast clouds,5.3833,-52.9500,
20,Sao Jose Da Coroa Grande,BR,77.49,few clouds,-8.8978,-35.1478,
25,Atuona,PF,78.22,clear sky,-9.8000,-139.0333,
28,Lorengau,PG,81.37,overcast clouds,-2.0226,147.2712,
30,Saldanha,ZA,77.05,scattered clouds,-33.0117,17.9442,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
# 7. Drop the rows where there is no Hotel Name.
WeatherPy_Vacation = hotel_df.dropna()
WeatherPy_Vacation.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Bosaso,SO,76.98,few clouds,11.2842,49.1816,Red Sea Hotel
6,Salalah,OM,75.29,clear sky,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
12,Hithadhoo,MV,82.60,clear sky,-0.6000,73.0833,Scoop Guest House
14,Sur,OM,77.76,clear sky,22.5667,59.5289,Sur Plaza Hotel
15,Rikitea,PF,77.32,overcast clouds,-23.1203,-134.9692,People ThankYou
18,Sinnamary,GF,75.15,overcast clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
20,Sao Jose Da Coroa Grande,BR,77.49,few clouds,-8.8978,-35.1478,Crôa Mares Hotel
25,Atuona,PF,78.22,clear sky,-9.8000,-139.0333,Villa Enata
28,Lorengau,PG,81.37,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
30,Saldanha,ZA,77.05,scattered clouds,-33.0117,17.9442,Blue Bay Lodge


In [22]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "../Vacation_Search/WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
WeatherPy_Vacation.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in WeatherPy_Vacation.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = WeatherPy_Vacation[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
locations = WeatherPy_Vacation[["Lat", "Lng"]]
max_temp = WeatherPy_Vacation["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))